# **🚀 Множественная диспетчеризация (Multiple dispatch) и**
# **Мультиметоды (Multimethods) в Julia.  Урок Девятый.**

<br>

## **📌 Темы:**

1. **Что такое множественная диспетчеризация (Multiple Dispatch)?**
   
2. **Что такое Multimethods?**
   

<br>

---

<br>

### **1️⃣ Что такое множественная диспетчеризация (Multiple Dispatch)?** 

**Множественная диспетчеризация (Multiple Dispatch)** — это механизм, позволяющий языку программирования **выбирать конкретную версию функции** в зависимости от **типа всех аргументов**, переданных в вызове.  

В отличие от традиционного **динамического полиморфизма (single dispatch)**, который основывается только на **первом аргументе (например, объекте класса в ООП)**, **множественная диспетчеризация учитывает все аргументы функции**, что делает код более гибким, расширяемым и производительным.

🔥 **Julia** — один из немногих языков, в которых **множественная диспетчеризация является основным механизмом работы с функциями**.

<br>

#### **Чем Multiple Dispatch отличается от Multimethods?**  

Хотя эти понятия связаны, они не полностью идентичны:  

| **Термин**             | **Определение** |
|------------------------|----------------|
| **Multiple Dispatch**  | Механизм выбора **правильного метода** функции **на основе всех аргументов**. Это ключевая особенность Julia. |
| **Multimethods**       | Организация кода с несколькими реализациями одной и той же функции, которые выбираются с помощью **множественной диспетчеризации**. |

💡 **Можно сказать, что мультиметоды — это практическое применение множественной диспетчеризации.**  

#### **🔹 Начинаем со знакомого.**

Чтобы понять множественную диспетчеризацию в Julia, начнем с того, что мы уже видели.

Мы можем объявлять функции в Julia, не предоставляя Julia никакой информации о типах входных аргументов, которые получит функция:

In [6]:
f(x) = x.^2

f (generic function with 1 method)

и тогда Julia самостоятельно определит, какие типы входных аргументов имеют смысл, а какие нет:

In [7]:
f(10)

100

In [8]:
f([1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

#### **🔹 Указание типов наших входных аргументов.**

Однако у нас также есть *возможность* явно указать Julia, какие типы могут иметь наши входные аргументы.

Например, давайте напишем функцию `foo`, которая принимает только строки в качестве входных данных.

In [11]:
foo(x::String, y::String) = println("Мои входные данные x и y — оба строки!")

foo (generic function with 1 method)

Здесь, для того, чтобы ограничить тип `x` и `y` до `String`, мы просто добавляем после имени входного аргумента двойное двоеточие `::` и ключевое слово `String`.

Мы знаем, что `foo` ,будет работать со `String` и не будет работать с другими типами входных аргументов.

In [14]:
foo("Привет!", "Hello!")

Мои входные данные x и y — оба строки!


In [15]:
foo(3, 4)  # ошибка

MethodError: MethodError: no method matching foo(::Int64, ::Int64)
The function `foo` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  foo(!Matched::String, !Matched::String)
   @ Main c:\Users\Siergej Sobolewski\Kursy Cartesian School\Julia\Introduction-to-Julia-main\Introduction-to-Julia-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y135sZmlsZQ==.jl:1


**Давайте изменим тип аргументов так, чтобы `foo` работал с целочисленными входными данными `Int`, т.е. добавим `::Int` к нашим входным аргументам при объявлении `foo`.**

In [16]:
foo(x::Int, y::Int) = println("Мои входные данные x и y — оба целые числа!")
foo(3, 4)  # теперь работает

Мои входные данные x и y — оба целые числа!


**Теперь `foo` работает с целыми числами! Но смотрите, `foo` также работает, когда `x` и `y` являются строками!**

In [18]:
foo("<Большой Привет", "Hello!")

Мои входные данные x и y — оба строки!



#### 🔹 **Начинаем добираться до сути множественной диспетчеризации.**

**Когда мы первый раз объявили:**

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

```
**А во второй раз мы объявили:**

```julia
foo(x::String, y::String)

```

**Мы не перезаписали и не заменили функцию `foo` !!!**

Вместо этого мы просто добавили дополнительный ***метод*** к ***универсальной функции*** под названием `foo`.

<br>

- ***Универсальная функция*** — это абстрактное понятие, связанное с определенной операцией.

Например, универсальная функция `+` представляет собой понятие сложения.


- ***метод*** — это конкретная реализация универсальной функции для *определенных типов аргументов*.

Например, `+` имеет методы, которые принимают числа с плавающей точкой, целые числа, матрицы и т. д.

<br>

Мы можем использовать `methods`, чтобы увидеть, сколько методов существует для `foo`:

In [19]:
methods(foo)

# 2 methods for generic function "foo" from Main:
 [1] foo(x::Int64, y::Int64)
     @ c:\Users\Siergej Sobolewski\Kursy Cartesian School\Julia\Introduction-to-Julia-main\Introduction-to-Julia-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y143sZmlsZQ==.jl:1
 [2] foo(x::String, y::String)
     @ c:\Users\Siergej Sobolewski\Kursy Cartesian School\Julia\Introduction-to-Julia-main\Introduction-to-Julia-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y135sZmlsZQ==.jl:1

**В данном сообщении мы видим информацию о том, что функция `foo` имеет два различных метода. Это означает, что при передаче разных аргументов Julia автоматически выбирает соответствующую версию функции с тем же именем, используя механизм множественной диспетчеризации (Multiple Dispatch).**

<br>

Итак, теперь мы можем вызывать `foo` для целых чисел или строк. Когда вы вызываете `foo` для определенного набора аргументов, Julia выведет типы входных данных и отправит соответствующий метод. *Это* и есть множественная  диспетчеризация.

Множественная  диспетчеризация делает наш код универсальным и быстрым. Наш код может быть универсальным и гибким, потому что мы можем писать код в терминах абстрактных операций, таких как сложение и умножение, а не в терминах конкретных реализаций. В то же время наш код выполняется быстро, потому что Julia может вызывать эффективные методы для соответствующих типов.

<br>

#### **🔹 Макрос `@which` поможет определить, какой же метод отправляется при вызове универсальной функции.**

Чтобы увидеть, какой метод отправляется при вызове универсальной функции, мы можем использовать макрос `@which` следующим образом:

In [20]:
@which foo(3, 4)

foo(x::Int64, y::Int64)
     @ Main c:\Users\Siergej Sobolewski\Kursy Cartesian School\Julia\Introduction-to-Julia-main\Introduction-to-Julia-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y143sZmlsZQ==.jl:1

Мы также можем добавить резервный метод с типом duck для `foo`, который принимает входные данные любого типа:

In [ ]:
foo(x, y) = println("Я принимаю любые предложения! :) ")

**Учитывая методы, которые мы уже написали для `foo`, этот метод будет вызываться всякий раз, когда мы передаем в `foo` нечисловые данные:**

In [24]:
v = rand(3)
foo(v, v)

Я принимаю любые предложения! :) 



📌 **Вот ещё один пример мультиметодов в Julia**:

In [21]:
# Определяем несколько версий функции process для разных типов аргументов
process(x::Int, y::Int) = x + y
process(x::Float64, y::Float64) = x * y
process(x::String, y::String) = x * " and " * y

# Вызовы функций
println(process(2, 3))           # 5 (Int)
println(process(2.5, 3.5))       # 8.75 (Float64)

println(process("Hello", "World"))  # "Hello and World" (String)
println(@which(process("Hello", "World")))  # process(x::String, y::String)

5
8.75
process(x::String, y::String) @ Main c:\Users\Siergej Sobolewski\Kursy Cartesian School\Julia\Introduction-to-Julia-main\Introduction-to-Julia-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y100sZmlsZQ==.jl:4


👉 **Что произошло?**

В коде выше мы видим, что Julia использует Multiple Dispatch, чтобы выбрать правильную версию функции `process()` в зависимости от типов аргументов.  

<br>

#### **🔹 Как Julia выбирает метод?**  

Когда мы вызываем функцию, Julia выполняет следующие шаги:  

**1️⃣ Определяет типы всех аргументов** в момент вызова.  

**2️⃣ Выбирает наиболее специфичный метод**, соответствующий данным типам.
  
**3️⃣ Компилирует оптимизированный код** для этого метода (при первом вызове).  

**Пример:**

In [2]:
multiply(a::Int, b::Int) = a * b 
multiply(a::Int, b::Float64) = a * b + 0.1
multiply(a::Float64, b::Float64) = a * b + 0.2

println(multiply(3, 4))       # 12 (Int)
println(multiply(3, 4.0))     # 12.1 (Int * Float64)
println(multiply(3.0, 4.0))   # 12.2 (Float64 * Float64)

12
12.1
12.2


🎯 **Julia автоматически выбирает наиболее подходящий метод на основе типов аргументов!**  

<br>

#### **🔹 Как Julia использует множественную диспетчеризацию в ООП?**  

Julia позволяет использовать **абстрактные типы** и **структуры**, что делает код еще более универсальным.

In [3]:
abstract type Animal end  # Абстрактный тип

struct Dog <: Animal
    name::String
end

struct Cat <: Animal
    name::String
end

speak(a::Dog) = println("Woof! I'm $(a.name)!")
speak(a::Cat) = println("Meow! I'm $(a.name)!")

dog = Dog("Buddy")
cat = Cat("Whiskers")

speak(dog)  # Woof! I'm Buddy!
speak(cat)  # Meow! I'm Whiskers!

Woof! I'm Buddy!
Meow! I'm Whiskers!


✅ Julia диспетчеризирует вызов `speak()` в зависимости от конкретного типа (`Dog` или `Cat`).

👉 **В отличие от ООП-языков (Java, Python, C#), где используется **single dispatch (по первому аргументу)**, Julia **учитывает все аргументы**.**

<br>

### **🔹 Использование `Union` для обобщенных методов**  

Если нужно объединить несколько типов в одном методе, можно использовать `Union`:


In [4]:
squared(x::Union{Int, Float64}) = x^2

println(squared(3))     # 9
println(squared(2.5))   # 6.25

9
6.25


✅ **Этот подход удобен, если разные типы данных должны обрабатываться одинаково.**

<br>


### **🔹 Метод `where` и параметрический полиморфизм**  

Julia поддерживает **параметрические методы**, позволяя писать код для **широкого класса типов**.


In [5]:
generic_add(a::T, b::T) where T = a + b

println(generic_add(10, 20))      # 30 (Int)
println(generic_add(2.5, 3.5))    # 6.0 (Float64)

30
6.0


✅ **Этот метод работает для любого типа `T`, если оба аргумента имеют один и тот же тип.**

<br>

#### **📌 Почему множественная диспетчеризация так важна? Что она нам даёт?**

- **Гибкость** — позволяет писать **универсальные функции**, работающие с разными комбинациями типов.

- **Производительность** — Julia компилирует код под **конкретные типы**, устраняя накладные расходы динамической типизации.  

- **Читаемость** — код остается **чистым и выразительным**, без сложных `if-else` и проверок типов.  

- **Модульность** — можно легко **расширять функциональность** без изменения исходного кода.  


#### **📌 Сравнение с другими языками** 

| **Язык**      | **Тип диспетчеризации** |
|--------------|----------------------|
| **Python**   | Single Dispatch (метод определяется по первому аргументу `self`) |
| **C++**      | Static Dispatch (решается во время компиляции) |
| **Java**     | Single Dispatch (выбор метода по типу объекта) |
| **C#**       | Single Dispatch + Dynamic Dispatch через `dynamic` |
| **Julia**    | **Multiple Dispatch (использует все аргументы)** |

🔥 **Julia выигрывает в гибкости и скорости, так как диспетчеризация выполняется на этапе компиляции, а не во время выполнения!** 🚀  


<br>

#### **📌 Где еще используется Multiple Dispatch?**  

✅ **Научные вычисления** – например, в линейной алгебре, где функции должны работать с разными типами чисел (Int, Float64, Complex).

✅ **Машинное обучение** – обработка данных, где алгоритмы применяются к различным типам входных данных.

✅ **Физическое моделирование** – выбор оптимального метода для расчета в зависимости от типов данных.  


<br>

---

<br>

### **2️⃣ Что такое Multimethods?**

Термин **"Multimethods"** в программировании можно перевести на русский язык как **"мультиметоды"** или **"многометодность"**.  

🔹 **Варианты перевода в зависимости от контекста:**  
- **"Мультиметоды"** – если речь идет о конкретной концепции в языке программирования.  
- **"Многометодность"** – если рассматривается как общая парадигма программирования.  
- **"Множественная диспетчеризация"** – если акцент на механизме выбора метода на основе нескольких аргументов.  


**Мультиметоды** – это механизм, позволяющий определять поведение функций (методов) в зависимости от **типа нескольких аргументов**. В отличие от традиционного **полиморфизма на основе одного аргумента** (как в ООП), мультиметоды позволяют **выбирать реализацию функции в зависимости от нескольких аргументов одновременно**.  

🔹 **Пример в Julia (язык с поддержкой мультиметодов):**  

In [2]:
function action(x::Int, y::Int)
    println("Оба аргумента целые числа: ", x + y)
end

function action(x::String, y::String)
    println("Оба аргумента строки: ", x * y)
end

action(2, 3)        # Оба аргумента целые числа: 5
action("Hi", "!")   # Оба аргумента строки: Hi!
action("Hello", 3)  # Ошибка, подходящего метода нет


Оба аргумента целые числа: 5
Оба аргумента строки: Hi!


MethodError: MethodError: no method matching action(::String, ::Int64)
The function `action` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  action(::String, !Matched::String)
   @ Main c:\Users\Siergej Sobolewski\Kursy Cartesian School\Julia\Introduction-to-Julia-main\Introduction-to-Julia-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y100sZmlsZQ==.jl:5
  action(!Matched::Int64, ::Int64)
   @ Main c:\Users\Siergej Sobolewski\Kursy Cartesian School\Julia\Introduction-to-Julia-main\Introduction-to-Julia-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y100sZmlsZQ==.jl:1


🔹 **Где применяются мультиметоды?**

- Языки с динамической типизацией (**Julia, Clojure, Common Lisp**).  
  
- Программирование на основе диспетчеризации типов. 
  
- Оптимизация кода для работы с разными типами данных.  

💡 **Вывод**: 

Мультиметоды позволяют создавать более **гибкие и адаптивные** программы, особенно в языках с динамической типизацией, таких как **Julia**.

<br>

---

<br>

## 🔹 **Упражнения:**

#### ✅ Задание 9.1

Кстати: как вы думаете, сколько существует методов сложения в Julia?

In [4]:
# Ваш ответ:


Подсказка:

Впишите  - ` methods(+) `